### Importar librerías

In [1]:
import numpy as np
import matplotlib.pyplot as plt

### Definir parámetros y funciones

In [2]:
# Parámetros
rho = 1000  # Densidad del agua en kg/m^3
Cp = 4186   # Capacidad calorífica específica del agua en J/kg·°C
volume = 1000000  # Volumen del embalse en metros cúbicos (1 millón m^3)
Q_solar = 500  # Radiación solar en W/m²
Q_loss = 50    # Pérdida de calor por evaporación y radiación en W/m²
area = 10000   # Área de la superficie del embalse en m²
time_step = 3600  # Paso de tiempo en segundos (1 hora)
total_time = 86400  # Tiempo total de simulación en segundos (1 día)
initial_temperature = 10  # Temperatura inicial en °C

# Inicializar variables
time_points = np.arange(0, total_time, time_step)
temperatures = np.zeros(len(time_points))
temperatures[0] = initial_temperature

# Función para calcular el cambio de temperatura
def temperature_change(Q_solar, Q_loss, volume, rho, Cp, area, dt):
    # Calcular el calor recibido (W)
    heat_input = Q_solar * area * dt
    # Calcular la pérdida de calor (W)
    heat_loss = Q_loss * area * dt
    # Aplicar la ecuación de balance de calor
    dT = (heat_input - heat_loss) / (volume * rho * Cp)
    return dT

# Simular los cambios de temperatura
for t in range(1, len(time_points)):
    dT = temperature_change(Q_solar, Q_loss, volume, rho, Cp, area, time_step)
    temperatures[t] = temperatures[t - 1] + dT

# Funciones biológicas
def growth_temperature(temp, temp_opt=15, sigma=2):
    return np.exp(-((temp - temp_opt) ** 2) / (2 * sigma ** 2))

def nutrient_limitation(np_ratio, k_np=86.6):
    return np_ratio / (k_np + np_ratio)

def light_limitation(radiation, max_radiation=80):
    return min(1, radiation / max_radiation)

def chlorophyll_a_production(temp, radiation, np_ratio, max_chlorophyll=40):
    temp_effect = growth_temperature(temp)
    light_effect = light_limitation(radiation)
    nutrient_effect = nutrient_limitation(np_ratio)
    return max_chlorophyll * min(temp_effect, light_effect, nutrient_effect)

### Ejecución

In [3]:
# === Pedir datos al usuario ===
np_ratio = float(input("Ingrese la relación N/P (mg/L): "))
radiation = float(input("Ingrese la radiación esperada (W/m²): "))

# Ajustar temperatura
temp_inicial = initial_temperature
temp_ajustada = temperatures.mean()

# Calcular producción de clorofila
produccion_clorofila = chlorophyll_a_production(temp_ajustada, radiation, np_ratio)

# === Mostrar resultados ===
print("\n--- Resultados ---")
print(f"Temperatura inicial: {temp_inicial:.2f} °C")
print(f"Temperatura ajustada: {temp_ajustada:.2f} °C")
print(f"Producción estimada de Clorofila-a: {produccion_clorofila:.2f} µg/L")

Ingrese la relación N/P (mg/L):  80
Ingrese la radiación esperada (W/m²):  80



--- Resultados ---
Temperatura inicial: 10.00 °C
Temperatura ajustada: 10.04 °C
Producción estimada de Clorofila-a: 1.86 µg/L


### Gráficas

In [ ]:
# === Graficar efectos ===
temperatures = np.linspace(temp_inicial - 5, temp_inicial + 5, 100)
chlorophyll_values = [chlorophyll_a_production(temp, radiation, np_ratio) for temp in temperatures]

plt.figure(figsize=(8, 5))
plt.plot(temperatures, chlorophyll_values, label="Clorofila-a vs Temperatura", color="green")
plt.axvline(temp_ajustada, color="red", linestyle="--", label=f"Temp ajustada: {temp_ajustada:.2f} °C")
plt.xlabel("Temperatura (°C)")
plt.ylabel("Clorofila-a (µg/L)")
plt.title("Producción de Clorofila-a según Temperatura")
plt.legend()
plt.grid()
plt.show()